# Data Load

## 1. 실습을 위해 torch 의 도구들 import

In [ ]:
from torch.utils.data import Dataset, DataLoader # Dataset, DataLoader import
# Dataset : custom dataset 에서 Dataset class 안에 있는 __getitem__() 과 __len__() 수정하여 사용하기 위함
# DataLoader : mini batch 단위로 데이터들을 iterable 하게 쪼개고 각 batch를 무작위로 섞게 해주는 class
from torch import from_numpy, tensor 
# from_numpy : numpy.ndarray(N차원의 배열 객체로, 같은 type 의 데이터로 구성) 로부터 Tensor 생성하도록 해주는 모듈
# tensor : Tensor 생성하도록 해주는 모듈
import numpy as np
# numpy : np.loadtxt 를 사용하기 위해 import

## 2. 모델 선언
### Dataset 의 subclass 인 custom dataset 생성 

In [ ]:
class DiabetesDataset(Dataset): # Dataset 의 subclass
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        xy = np.loadtxt('./data/diabetes.csv.gz',
                        delimiter=',', dtype=np.float32) 
        # xy : csv 파일의 데이터를 float type array 로 저장
        # delimiter : csv가 콤마(,)로 행,열을 구분하므로 각 데이터를 delimiter=',' 로  쪼개준다.
        # dtype=float32 : 데이터들을 float32 로 형변환해준다

        self.len = xy.shape[0] # len : xy의 행 개수
        self.x_data = from_numpy(xy[:, 0:-1])  # 첫번째 :->  행 전체 선택, 두번째 :-> 전체에서 가장 우측 열 제외한 나머지 선택
        self.y_data = from_numpy(xy[:, [-1]]) # 첫번째 :->  행 전체 선택, [-1] -> 가장 우측 열만 선택

    def __getitem__(self, index): # 인덱스를 받음
        return self.x_data[index], self.y_data[index] # __getitem__ : 주어진 index 에 맞는 item 을 반환

    def __len__(self): # object의 행 개수 
        return self.len # 반환

## 3. custom dataset 의 instance 생성

In [ ]:
dataset = DiabetesDataset()

## 4. train_loader 에 DataLoader 사용하여 초기화


In [ ]:
train_loader = DataLoader(dataset=dataset, # DataLoader 는 기본적으로 2개의 인자(데이터셋, 미니 배치의 크기)를 입력받음
                          batch_size=32,   # 미니 배치의 크기는 보통 2의 배수를 사용 ( default : batch_size = 1 )
                          shuffle=True,    # 추가적으로 shuffle 인자 사용 (default : shuffle = False)
                                                            # -> shuffle=True 하면 데이터 학습 순서 무작위
                                                            # < shuffle=True 사용 이유 >
                                                            # 매 epoch 마다 똑같은 순서로 학습하면 
                                                            # dataset 내용물 보다 dataset의 순서를 학습하여 바람직하지 않음
                          num_workers=2)    # num_workers : multiple processes 를 필요로 할 때 사용

## 5. 훈련

In [ ]:
for epoch in range(2):
     # train_loader 는 iterable 하므로 for문을 돌릴 수 있다. == 미니 배치를 차례로 data에 저장가능하다
    for i, data in enumerate(train_loader, 0): # i : batch의 index 저장, data : train_loader가 가져온 미니 배치 저장
        # get the inputs
        inputs, labels = data # mini batch 들을 inputs 과 labels 로 나눈다

        # wrap them in Variable
        inputs, labels = tensor(inputs), tensor(labels) 

        # Run your training process
        print(f'Epoch: {i} | Inputs {inputs.data} | Labels {labels.data}')